# `pygvec` example for using B-spline profiles for a tokamak

This notebook re-uses the *gvecrun_tokamak* example to demonstrate how `pygvec` interfaces with the B-spline radial plasma profiles. The first three cells just provide some utility for running gvec in this notebook.

In [23]:
import os
from pathlib import Path
import contextlib
import shutil

import numpy as np
import matplotlib.pyplot as plt

# OMP number of threads for gvec run needs to be before import of gvec
os.environ["OMP_NUM_THREADS"]="2"
# needs `pip install` of gvec in virtual environment, and to be run in that environment!!!
import gvec  # using run & modifying the parameters & postprocessing

In [24]:
 @contextlib.contextmanager
def chdir(path: Path | str):
    """
    Contextmanager to change the current working directory.

    Using a context has the benefit of automatically changing back to the original directory when the context is exited, even if an exception is raised.
    """
    path = Path(path)
    old_dir = Path(os.getcwd())

    os.chdir(path)
    yield
    os.chdir(old_dir)

In [25]:
template = "parameter.ini"
runpath = Path(f"run_{1:02d}")

if not runpath.exists():
    runpath.mkdir()
    print(f"created run directory {runpath}")

Now we want to interpolate some pressure profile shape and hand it over to `gvec`. First we define a simple profile parametrisation, a two-power profile:

In [ ]:
def two_power(s, a=3, b=5):
    return (1-s**a)**b

# first derivative with respect to rho=sqrt(s)
def two_power_dr(s, a=3, b=5):
    return b*(1-s**a)**(b-1)*(-a*s**(a-1))*2*np.sqrt(s)

# second derivative with respect to rho=sqrt(s)
def two_power_drr(s, a=3, b=5):
    return 2*a*b*s**(a-1)*(1-s**a)**(b-2)*(2*a*(b*s**a-1)-s**a+1)

x_tp = np.linspace(0,1,20)
y_tp = two_power(x_tp)

x_plot = np.linspace(0,1,100)
y_plot = two_power(x_plot)

fig, ax = plt.subplots()
ax.plot(x_plot,y_plot,label="two power profile")
ax.plot(x_tp, y_tp, "o", label="interpolation points")
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()

Now we want to adapt an existing parameter file with some changed values and this two-power pressure profile:

In [ ]:
params= {}
iota_coefs=[0.523,0.625]  # a0*s + a1

# This profile uses the default polynomial representation
params["iota_coefs"] = gvec.util.np2gvec(iota_coefs) # c_0 + c_1*s +c_2*s^2+ ...
params["sgrid_nElems"] = 5
params["pres_scale"] = 1500
params["maxiter"] = 1000
params

By handing our defined interpolation points to `gvec.util.profile2bspl` we can generate the B-spline interpolation. Alternatively we coud just hand over the `two_power` callable, possibly with predefined s-interpolation points. Per default the interpolation is done using splines of degree 3 and 10 linearly spaced s-positions. 

In [ ]:
params = gvec.util.profile2bspline("pres", y_tp, x_tp, params=params) # handing over the params dict extends it
params

These parameters can then be handed to `gvec.util.adapt_parameter_file` or we could directly provide the source and target file.

In [ ]:
params = gvec.util.profile2bspline("pres", y_tp, x_tp, params=params, deg=4,
                                   source=template, target=runpath / "parameter.ini")
params

With the new parameters we can thus run `gvec` and evaluate the profile and its derivatives with respect to $\rho=\sqrt{s}$.

In [30]:
gvec.util.adapt_parameter_file(template, runpath / "parameter.ini", **params)

with chdir(runpath):
    gvec.run("parameter.ini", stdout_path="stdout.txt")

In [31]:
statefile = sorted(runpath.glob("*State*.dat"))[-1]
with gvec.State(runpath / "parameter.ini", statefile) as state:
    rho = np.linspace(0, 1, 100)  # radial visualization points
    p = state.evaluate_profile("p",rho=rho)
    dp_dr = state.evaluate_profile_deriv("p",rho=rho, n=1)
    dp_drr = state.evaluate_profile_deriv("p",rho=rho, n=2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot,y_plot,label="two power profile")
ax.plot(x_tp, y_tp, "o", label="interpolation points")
ax.plot(rho**2, p/p[0], "r--", label="gvec B-spline") # note that the pressure profile is scaled with pres_scale
ax.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax.set_ylabel(r"$P/P_0$")
plt.legend()

In [ ]:
fig_dr, ax_dr = plt.subplots()
ax_dr.plot(x_plot,two_power_dr(x_plot),label="two power profile")
ax_dr.plot(rho**2, dp_dr/p[0], "r--", label="gvec B-spline") # note that the pressure profile is scaled with pres_scale
ax_dr.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax_dr.set_ylabel(r"$P'/P_0$")
plt.legend()

fig_drr, ax_drr = plt.subplots()
ax_drr.plot(x_plot,two_power_drr(x_plot),label="two power profile")
ax_drr.plot(rho**2, dp_drr/p[0], "r--", label="gvec B-spline") # note that the pressure profile is scaled with pres_scale
ax_drr.set_xlabel(r"s = $\frac{\phi}{\phi_{norm}}$")
ax_drr.set_ylabel(r"$P''/P_0$")
plt.legend()